It seems likely that for our project, we'll need some kind of dice-rolling mechanism, since it takes inspiration from the rules of D&D. 

We have a subroutine that generates random numbers, courtesy of dblank: 

In [12]:
.ORIG x3000
;;; Algorithm for the iteration xn <- a x(n-1) mod m
;;; using Schrage's method

    JSR Random
    ;; look in R1 for a pseudo-random value; call Random again to iterate through them
    JSR Random
    ;; look in R1 for a pseudo-random value; call Random again to iterate through them
    JSR Random
    ;; look in R1 for a pseudo-random value; call Random again to iterate through them
    JSR Random
    ;; look in R1 for a pseudo-random value; call Random again to iterate through them
    HALT

;;; -----------------------------------------------------
;;; Memory X has next random number
Random: ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R7, BACK         ; Restore return address
    RET
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  RET            ; R2 has the sum
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


This subroutine generates random numbers in the whole range of a register. What do we do if we want a D8 or a D20? Well, a D8 is relatively simple: we can take only the last three bits of the output, using a mask, and get a random number between zero and seven (and then add one). 

Numbers that are not powers of two are harder. The first idea to occur is to use the modulo operator to map the integers onto Z20, but no power of two will be evenly divisible by 20, so the distribution will never be quite right.

But probably taking numbers between 0 and 63 and modding them by 20 is close enough? 

In [21]:
.ORIG x3000

        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D20
        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D20
        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D20
    HALT

;; ------------------------------------------------------
;; Function to generate a number not-quite-evenly-distributed between 1 and 20
D20: 
    ;; First call Random to generate a random number with the full register range. Random doesn't use the stack, so we won't either
    JSR RANDOM 
    
    LD R1, X            ;; Load in random number (might have to use LDI if these end up far apart)
    LD R2, MASK
    AND R1, R1, R2      ;; Now R1 is between 0 and 31
    AND R0, R0, #0
    ADD R0, R0, R1
    AND R1, R1, #0
    ADD R1, R1, #20
    
    ;; Call Divide to get R0/20 into R6 (Again not using the stack bc DIV doesn't)
    JSR Divide
    LD R6, RESULT
    
    ;; Pop stack
    LD R6, Top
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R7, R6, #0      ;; Load stack into R7
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R5, R6, #0      ;; Load Stack into R5
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R4, R6, #0      ;; Load Stack into R4
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R3, R6, #0      ;; Load Stack into R3
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R2, R6, #0      ;; Load Stack into R2
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R1, R6, #0      ;; Load Stack into R1
    ADD R6, R6, #-1     ;; Decrement Stack
    LDR R0, R6, #0      ;; Load Stack into R0
    ST R6, Top
    RET
    
RESULT: .FILL #0
MASK: .FILL x3F
    


;;; -----------------------------------------------------
;;; Memory X has next random number
Random: ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R7, BACK         ; Restore return address
    RET
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  RET            ; R2 has the sum
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

x30B4


In [22]:
%dis

Memory disassembled:
           x3000: x2C00  LD R6, x3001                              [line: 2]
           x3001: x7180  STR R0, R6, 0                             [line: 3]
           x3002: x1DA1  ADD R6, R6, #1                            [line: 4]
           x3003: x7380  STR R1, R6, 0                             [line: 5]
           x3004: x1DA1  ADD R6, R6, #1                            [line: 6]
           x3005: x7580  STR R2, R6, 0                             [line: 7]
           x3006: x1DA1  ADD R6, R6, #1                            [line: 8]
           x3007: x7780  STR R3, R6, 0                             [line: 9]
           x3008: x1DA1  ADD R6, R6, #1                            [line: 10]
           x3009: x7980  STR R4, R6, 0                             [line: 11]
           x300A: x1DA1  ADD R6, R6, #1                            [line: 12]
           x300B: x7B80  STR R5, R6, 0                             [line: 13]
           x300C: x1DA1  ADD R6, R6, #1            

In [2]:
%exe

Computation completed
Instructions: 52
Cycles: 432 (0.000216 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x7187 R7: x3034 


**Attempting to modify code to use D8.**

Tried putting in the new stack code, but then there started being bugs that I'm having trouble finding. I'm thinking it might have something to do with how the registers are used between Random and the current version of the stack, but I could be super wrong about that. When using the new stack the program assembles and sometimes runs with a NOOP error or just doesn't run at all.

In [19]:
.ORIG x3000

BRnzp D8

Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET
        
    HALT

;; Roll an 8-sided die

D8: 
    ;; First call Random to generate a random number with the full register range. Random doesn't use the stack, so we won't either
    
    JSR Random 
    
    LD R1, X            ;; Load in random number (might have to use LDI if these end up far apart)
    LD R2, MASK
    AND R1, R1, #7
    ADD R1, R1, #1
    
    JSR POP
    
RESULT: .FILL #0
MASK: .FILL #7
    
;;; Memory X has next random number

Random: 
    ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  
    BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: 
    ST R2, X
    LD R7, BACK         ; Restore return address
    RET
    
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  RET            ; R2 has the sum
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [12]:
%dis

Memory disassembled:
           x3000: x0E88  BRnzp D8 (or 136)                         [line: 2]
TOP:       x3001: x2FF4  LD R7, x2FF6                              [line: 4]
STACK:     x3002: x1DA1  ADD R6, R6, #1                            [line: 5]
           x3003: x7380 - 29568 
           x3004: x1DA1 - 7585 
           x3005: x7580 - 30080 
           x3006: x1DA1 - 7585 
           x3007: x7780 - 30592 
           x3008: x1DA1 - 7585 
           x3009: x7980 - 31104 
           x300A: x1DA1 - 7585 
           x300B: x7B80 - 31616 
           x300C: x1DA1 - 7585 
           x300D: x7F80 - 32640 
           x300E: x1DA1 - 7585 
           x300F: x3C00 - 15360 
           x3010: x0E8F - 3727 
           x3011: x2C00 - 11264 
           x3012: x7180 - 29056 
           x3013: x1DA1 - 7585 
           x3014: x7380 - 29568 
           x3015: x1DA1 - 7585 
           x3016: x7580 - 30080 
           x3017: x1DA1 - 7585 
           x3018: x7780 - 30592 
           x3019: x1DA1 - 7585 


In [20]:
%exe


Runtime error:
    memory x3005
Attempting to execute NOOP at x3005



Testing D20 with the new stack. Similar errors to above.

In [39]:
.ORIG x3000

BRnzp D20

Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;; ------------------------------------------------------
;; Function to generate a number not-quite-evenly-distributed between 1 and 20
D20: 
    ;; First call Random to generate a random number with the full register range. Random doesn't use the stack, so we won't either
    JSR Random 
    
    LD R1, X            ;; Load in random number (might have to use LDI if these end up far apart)
    LD R2, MASK
    AND R1, R1, R2      ;; Now R1 is between 0 and 31
    AND R0, R0, #0
    ADD R0, R0, R1
    AND R1, R1, #0
    ADD R1, R1, #20
    
    ;; Call Divide to get R0/20 into R6 (Again not using the stack bc DIV doesn't)
    JSR Divide
    LD R6, RESULT
    
    JSR POP
    
RESULT: .FILL #0
MASK: .FILL x3F
    


;;; -----------------------------------------------------
;;; Memory X has next random number
Random: ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R7, BACK         ; Restore return address
    RET
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  RET            ; R2 has the sum
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [41]:
%dis

Memory disassembled:
           x3000: x4889  JSR D20                                   [line: 2]
TOP:       x3001: x2FFB  LD R7, x2FFD                              [line: 4]
STACK:     x3002: x5267  AND R1, R1, #7                            [line: 5]
           x3003: x1261 - 4705 
           x3004: x3E26 - 15910 
           x3005: x201F - 8223 
           x3006: x221D - 8733 
           x3007: x483C - 18492 
           x3008: x205C - 8284 
           x3009: x301E - 12318 
           x300A: x205B - 8283 
           x300B: x301B - 12315 
           x300C: x2019 - 8217 
           x300D: x221A - 8730 
           x300E: x4835 - 18485 
           x300F: x2255 - 8789 
           x3010: x3219 - 12825 
           x3011: x2254 - 8788 
           x3012: x3216 - 12822 
           x3013: x2010 - 8208 
           x3014: x4817 - 18455 
           x3015: x3413 - 13331 
           x3016: x2010 - 8208 
           x3017: x2212 - 8722 
           x3018: x4813 - 18451 
           x3019: x94BF - 38079 
 

In [40]:
%exe

Keyboard Interrupt!


Testing D8 with old stack. It at least runs in this case, but the result should be showing up in R1 and it's not.

In [25]:
.ORIG x3000

        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D8
        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D8
        LD R6, Top
        STR R0, R6, #0      ;; Save R0 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R1, R6, #0      ;; Save R1 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R2, R6, #0      ;; Save R2 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R3, R6, #0      ;; Save R3 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R4, R6, #0      ;; Save R4 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R5, R6, #0      ;; Save R5 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        STR R7, R6, #0      ;; Save R7 to Stack
        ADD R6, R6, #1      ;; Increment Stack
        ST R6, Top
    JSR D8
    HALT

;; ------------------------------------------------------
;; Function to generate a number not-quite-evenly-distributed between 1 and 20
D8: 
    ;; First call Random to generate a random number with the full register range. Random doesn't use the stack, so we won't either
    
    JSR Random 
    
    LD R1, X            ;; Load in random number (might have to use LDI if these end up far apart)
    LD R2, MASK
    AND R1, R1, R2
    ADD R1, R1, #1
    
    JSR POP
    
RESULT: .FILL #0
MASK: .FILL #7
    


;;; -----------------------------------------------------
;;; Memory X has next random number
Random: ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R7, BACK         ; Restore return address
    RET
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  RET            ; R2 has the sum
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
.END

x309F


In [26]:
%exe

Computation completed
Instructions: 52
Cycles: 432 (0.000216 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x7187 R7: x3034 
